[https://medium.com/@ab.jannatpour/advanced-pairs-trading-kalman-filters-b171144ce6d5](https://)

In [1]:
import sys
sys.path.append('../../')

In [2]:
from util.load_data import *
from config.constants import *

In [3]:
XAUUSD = load_data()

In [5]:
data_1 = XAUUSD['D1']

In [4]:
data_2 = pd.read_csv(
    PATH_DATA + '/USDJPY/USDJPY_D1 - USDJPY_D1' + '.csv', index_col='Time', parse_dates=True)

In [8]:
# Define the system model for the spread
A = 1
Q = 0.1

# Define the measurement model for the spread
H = 1
R = 0.1

In [10]:
mu = 0
sigma = 0.1

In [12]:
x = 0
P = 1

In [13]:
import numpy as np
from pykalman import KalmanFilter
import pandas as pd

# loading stocks data
stock1 = data_1
stock2 = data_2

# calculate the spread
spread = stock1['Close'] - stock2['Close']

# create the kalman filter object
kf = KalmanFilter(transition_matrices=A, observation_matrices=H,
                  initial_state_mean=x, initial_state_covariance=P,
                  transition_covariance=Q, observation_covariance=R)

# perform the prediction and update steps
for i in range(100):
    # prediction
    x_pred, P_pred = kf.filter_update(
        filtered_state_mean=x, filtered_state_covariance=P)

    # measurement
    z = spread[i]

    # update
    x, P = kf.filter_update(filtered_state_mean=x_pred,
                            filtered_state_covariance=P_pred, observation=z)

In [21]:
# Perform the prediction and update steps for each time step in the historical data
(filtered_state_means, filtered_state_covariances) = kf.filter(spread.values[-1])

In [24]:
data_2

,Open,High,Low,Close,Volume
Time,,,,,
2007-02-12,121.900,121.953,121.730,121.764,64849
2007-02-13,121.760,121.766,121.090,121.263,618320
2007-02-14,121.272,121.347,120.226,120.286,616539
2007-02-15,120.282,120.359,119.147,119.376,617115
2007-02-16,119.370,119.550,118.984,119.392,586044
...,...,...,...,...,...
2023-03-06,136.016,136.190,135.364,135.995,506836
2023-03-07,135.995,137.438,135.537,137.387,595894
2023-03-08,137.392,137.910,136.477,137.223,528313


In [22]:
filtered_state_covariances.size

1